## Step 1 : Imports

In [1]:
import pandas as pd

## Step 2: Load the oof files

In [2]:
fold_df = pd.read_csv("/kaggle/input/folding-strategy-rna/rna_dataset_with_fold.csv")
fold_df = fold_df[fold_df["fold"]==1]
fold_df.shape

(178, 7)

In [3]:
fold_df.columns

Index(['target_id', 'sequence', 'temporal_cutoff', 'description',
       'all_sequences', 'sequence_length', 'fold'],
      dtype='object')

In [4]:
fold_df

,target_id,sequence,temporal_cutoff,description,all_sequences,sequence_length,fold
67,1FFK_9,UUAGGCGGCCACAGCGGUGGGGUUGCCUCCCGUACCCAUCCCGAAC...,2000-08-14,CRYSTAL STRUCTURE OF THE LARGE RIBOSOMAL SUBUN...,>1FFK_1|Chain A[auth 0]|23S RRNA|Haloarcula ma...,122,1
74,1FOQ_A,UCAAUGGUACGGUACUUCCAUUGUCAUGUGUAUGUUGGGGAUUAAA...,2000-12-22,PENTAMERIC MODEL OF THE BACTERIOPHAGE PHI29 PR...,">1FOQ_1|Chains A, B, C, D, E|BACTERIOPHAGE PHI...",120,1
148,1S9S_A,GGCGGUACUAGUUGAGAAACUAGCUCUGUAUCUGGCGGACCCGUGG...,2004-04-06,SOLUTION STRUCTURE OF MLV PSI SITE,>1S9S_1|Chain A|MLV Psi encapsidation site|nul...,101,1
173,1ZC8_Z,GCCCGUCACGCCAUGGGAGCGGGCUCUACCCGAAGUCGCCGGGAGC...,2005-04-19,"Coordinates of tmRNA, SmpB, EF-Tu and h44 fitt...",>1ZC8_1|Chain A|TLD 16S ribosomal RNA|Thermus ...,91,1
186,1YSH_B,UAGACGGUGGGAGAGGGUGGUGGAAUUCCCGGAGUAGCGGUGAAAU...,2005-07-05,Localization and dynamic behavior of ribosomal...,>1YSH_1|Chain A|RNA (28-MER)|synthetic constru...,101,1
...,...,...,...,...,...,...,...
832,9G7C_A,GGGAGUCGGGUCUGUGGUUGAAAGUCGAUGCCAGUCGCAGGCGAAA...,2024-10-30,Cryo-EM structure of raiA ncRNA from Clostridi...,>9G7C_1|Chain A|raiA (224-MER)|Clostridium sp....,224,1
839,8T3E_EC,AAACUCCAUGUAUUGGUUACCCAUCUGCAUCGAAAACUCUCCGAAC...,2024-12-11,Hypomethylated yeast 80S bound with Taura synd...,>8T3E_1|Chain A[auth BA]|40S ribosomal protein...,202,1
840,8T3F_EC,AAACUCCAUGUAUUGGUUACCCAUCUGCAUCGAAAACUCUCCGAAC...,2024-12-11,Hypomethylated yeast 80S bound with Taura synd...,>8T3F_1|Chain A[auth BA]|40S ribosomal protein...,202,1
842,8Z1G_T,GGUAAAAUGGCUGAGUGAAGCAUUGGACUGUAAAUCUAAAGACAGG...,2024-12-18,Cryo-EM structure of human ELAC2-pre-tRNA,>8Z1G_1|Chain A[auth T]|Homo sapiens mitochond...,86,1


In [5]:
exp_1_oof = pd.read_csv("/kaggle/input/protenix/proteinx_oof.csv")
exp_2_oof = pd.read_csv("/kaggle/input/train-ribonanzanet-inference-03/ribonanzanet1_oof.csv")
exp_3_oof = pd.read_csv("/kaggle/input/ribonanzanet-3d-inference/ribonanzanet1_public_oof.csv")
exp_4_oof = pd.read_csv("/kaggle/input/ribonanzanet2-ddpm-inference/ribonanzanet2_oof.csv")
exp_5_oof = pd.read_csv("/kaggle/input/train-rhofold-inference/RhoFold/rhofold_01_oof.csv")

In [6]:
exp_1_oof["target_id"] = exp_1_oof["ID"].apply(lambda x: x.split("_")[0]+'_'+x.split("_")[1])
exp_2_oof["target_id"] = exp_2_oof["ID"].apply(lambda x: x.split("_")[0]+'_'+x.split("_")[1])
exp_3_oof["target_id"] = exp_3_oof["ID"].apply(lambda x: x.split("_")[0]+'_'+x.split("_")[1])
exp_4_oof["target_id"] = exp_4_oof["ID"].apply(lambda x: x.split("_")[0]+'_'+x.split("_")[1])
exp_5_oof["target_id"] = exp_5_oof["ID"].apply(lambda x: x.split("_")[0]+'_'+x.split("_")[1])

In [7]:
exp_1_oof

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,target_id
0,1FFK_9_1,U,1,6.074,-27.246,-7.244,1.181,-28.209,5.954,-20.874,-4.576,14.949,-10.065,-20.620,7.468,17.033,-4.355,22.271,1FFK_9
1,1FFK_9_2,U,2,5.920,-24.237,-2.370,3.976,-23.706,9.493,-18.963,0.381,14.679,-15.418,-14.987,4.753,15.270,-8.090,18.702,1FFK_9
2,1FFK_9_3,A,3,0.383,-21.406,-1.219,-1.625,-18.410,10.248,-18.046,0.602,6.970,-12.298,-11.882,8.983,15.393,-5.057,12.270,1FFK_9
3,1FFK_9_4,G,4,7.584,-21.293,3.046,7.512,-19.914,10.996,-15.313,5.580,13.580,-6.862,-16.268,13.934,12.024,-11.729,15.557,1FFK_9
4,1FFK_9_5,G,5,11.798,-17.503,4.719,12.465,-17.354,9.293,-10.629,7.841,16.460,-1.181,-16.817,15.085,7.355,-15.107,15.093,1FFK_9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21017,8Z1F_T_82,U,82,-10.500,-42.947,30.896,24.340,-51.092,-22.515,33.472,-30.068,-34.494,-22.995,17.567,60.983,9.585,40.167,-49.065,8Z1F_T
21018,8Z1F_T_83,C,83,-10.124,-37.743,32.391,29.828,-51.314,-23.599,30.065,-31.815,-38.368,-28.158,18.131,61.958,13.068,40.807,-53.023,8Z1F_T
21019,8Z1F_T_84,A,84,-7.433,-33.068,33.569,34.977,-52.492,-22.113,28.445,-31.655,-43.460,-32.922,16.280,62.450,14.942,39.032,-57.524,8Z1F_T
21020,8Z1F_T_85,U,85,-3.329,-29.941,32.058,38.746,-55.271,-18.938,29.362,-30.660,-48.963,-36.571,12.427,63.402,14.045,36.491,-62.390,8Z1F_T


In [8]:
LABEL_DF = pd.read_csv("/kaggle/input/stanford-rna-3d-folding/train_labels.csv")
LABEL_DF

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,ID,resname,resid,x_1,y_1,z_1
0,1SCL_A_1,G,1,13.760,-25.974001,0.102
1,1SCL_A_2,G,2,9.310,-29.638000,2.669
2,1SCL_A_3,G,3,5.529,-27.813000,5.878
3,1SCL_A_4,U,4,2.678,-24.900999,9.793
4,1SCL_A_5,G,5,1.827,-20.136000,11.793
...,...,...,...,...,...,...
137090,8Z1F_T_82,U,82,NaN,NaN,NaN
137091,8Z1F_T_83,C,83,NaN,NaN,NaN
137092,8Z1F_T_84,A,84,NaN,NaN,NaN
137093,8Z1F_T_85,U,85,NaN,NaN,NaN


In [9]:
LABEL_DF.rename(columns={'x_1':'x_1_true','y_1':'y_1_true','z_1':'z_1_true'}, inplace=True)

In [10]:
LABEL_DF

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,ID,resname,resid,x_1_true,y_1_true,z_1_true
0,1SCL_A_1,G,1,13.760,-25.974001,0.102
1,1SCL_A_2,G,2,9.310,-29.638000,2.669
2,1SCL_A_3,G,3,5.529,-27.813000,5.878
3,1SCL_A_4,U,4,2.678,-24.900999,9.793
4,1SCL_A_5,G,5,1.827,-20.136000,11.793
...,...,...,...,...,...,...
137090,8Z1F_T_82,U,82,NaN,NaN,NaN
137091,8Z1F_T_83,C,83,NaN,NaN,NaN
137092,8Z1F_T_84,A,84,NaN,NaN,NaN
137093,8Z1F_T_85,U,85,NaN,NaN,NaN


In [11]:
exp_1_oof = exp_1_oof.merge(LABEL_DF[['x_1_true','y_1_true','z_1_true','ID']], on='ID', how='left')

In [12]:
exp_1_oof.shape, exp_2_oof.shape, exp_3_oof.shape, exp_4_oof.shape

((21022, 22), (59508, 24), (59508, 24), (25713, 22))

In [13]:
exp_1_oof.columns, exp_2_oof.columns, exp_3_oof.columns

(Index(['ID', 'resname', 'resid', 'x_1', 'y_1', 'z_1', 'x_2', 'y_2', 'z_2',
        'x_3', 'y_3', 'z_3', 'x_4', 'y_4', 'z_4', 'x_5', 'y_5', 'z_5',
        'target_id', 'x_1_true', 'y_1_true', 'z_1_true'],
       dtype='object'),
 Index(['Unnamed: 0', 'ID', 'resname', 'resid', 'x_1', 'y_1', 'z_1', 'x_2',
        'y_2', 'z_2', 'x_3', 'y_3', 'z_3', 'x_4', 'y_4', 'z_4', 'x_5', 'y_5',
        'z_5', 'flag', 'x_1_true', 'y_1_true', 'z_1_true', 'target_id'],
       dtype='object'),
 Index(['Unnamed: 0', 'ID', 'resname', 'resid', 'x_1', 'y_1', 'z_1', 'x_2',
        'y_2', 'z_2', 'x_3', 'y_3', 'z_3', 'x_4', 'y_4', 'z_4', 'x_5', 'y_5',
        'z_5', 'flag', 'x_1_true', 'y_1_true', 'z_1_true', 'target_id'],
       dtype='object'))

In [14]:
exp_1_oof.head()

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,...,x_4,y_4,z_4,x_5,y_5,z_5,target_id,x_1_true,y_1_true,z_1_true
0,1FFK_9_1,U,1,6.074,-27.246,-7.244,1.181,-28.209,5.954,-20.874,...,-10.065,-20.620,7.468,17.033,-4.355,22.271,1FFK_9,50.841000,14.916,99.295998
1,1FFK_9_2,U,2,5.920,-24.237,-2.370,3.976,-23.706,9.493,-18.963,...,-15.418,-14.987,4.753,15.270,-8.090,18.702,1FFK_9,52.405998,10.401,106.478996
2,1FFK_9_3,A,3,0.383,-21.406,-1.219,-1.625,-18.410,10.248,-18.046,...,-12.298,-11.882,8.983,15.393,-5.057,12.270,1FFK_9,57.773998,14.789,103.852997
3,1FFK_9_4,G,4,7.584,-21.293,3.046,7.512,-19.914,10.996,-15.313,...,-6.862,-16.268,13.934,12.024,-11.729,15.557,1FFK_9,57.311001,16.924,96.344002
4,1FFK_9_5,G,5,11.798,-17.503,4.719,12.465,-17.354,9.293,-10.629,...,-1.181,-16.817,15.085,7.355,-15.107,15.093,1FFK_9,57.646999,20.913,92.202003


In [15]:
# Find common target_ids across all three dataframes
common_target_ids = set(exp_1_oof['target_id']).intersection(
    set(exp_2_oof['target_id'])).intersection(
    set(exp_3_oof['target_id'])).intersection(
    set(exp_4_oof['target_id'])).intersection(
    set(exp_5_oof['target_id'])).intersection(
    set(fold_df['target_id']))

# Filter exp_1_oof to keep only rows with common target_ids
exp_1_oof = exp_1_oof[exp_1_oof['target_id'].isin(common_target_ids)]
exp_2_oof = exp_2_oof[exp_2_oof['target_id'].isin(common_target_ids)]
exp_3_oof = exp_3_oof[exp_3_oof['target_id'].isin(common_target_ids)]
exp_4_oof = exp_4_oof[exp_4_oof['target_id'].isin(common_target_ids)]
exp_5_oof = exp_5_oof[exp_5_oof['target_id'].isin(common_target_ids)]

In [16]:
len(common_target_ids)

161

In [17]:
exp_1_oof.shape, exp_2_oof.shape, exp_3_oof.shape, exp_4_oof.shape

((21022, 22), (19206, 24), (19206, 24), (21020, 22))

In [18]:
import pandas as pd

# Define key columns for matching
key_cols = ['ID', 'resname', 'resid', 'target_id']

# Find common keys across all three DataFrames
common_keys = pd.merge(exp_1_oof[key_cols], exp_2_oof[key_cols], on=key_cols)
common_keys = pd.merge(common_keys, exp_3_oof[key_cols], on=key_cols)
common_keys = pd.merge(common_keys, exp_4_oof[key_cols], on=key_cols)
common_keys = pd.merge(common_keys, exp_5_oof[key_cols], on=key_cols)

# Filter each DataFrame using common keys
exp_1_oof = pd.merge(exp_1_oof, common_keys, on=key_cols)
exp_2_oof = pd.merge(exp_2_oof, common_keys, on=key_cols)
exp_3_oof = pd.merge(exp_3_oof, common_keys, on=key_cols)
exp_4_oof = pd.merge(exp_4_oof, common_keys, on=key_cols)
exp_5_oof = pd.merge(exp_5_oof, common_keys, on=key_cols)

In [19]:
exp_1_oof.shape, exp_2_oof.shape, exp_3_oof.shape, exp_4_oof.shape, exp_5_oof.shape

((19206, 22), (19206, 24), (19206, 24), (19206, 22), (19206, 23))

In [20]:
exp_1_oof.attrs["name"] = "PROTEINX"
exp_2_oof.attrs["name"] = "RINET1"
exp_3_oof.attrs["name"] = "RINET0"
exp_4_oof.attrs["name"] = "RINET2"
exp_5_oof.attrs["name"] = "RhoFold"

## Step -1: Hill climbing code

In [21]:
import gc
import os
import re
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing

In [22]:
def setup_usalign():
    """
    Sets up USalign by copying it to the working directory and making it executable.
    """
    # Copy USalign to working directory if it doesn't exist
    if not os.path.exists('/kaggle/working/USalign'):
        os.system('cp //kaggle/input/usalign/USalign /kaggle/working/')
        os.system('sudo chmod u+x /kaggle/working/USalign')
        print("USalign binary copied and made executable")
    return '/kaggle/working/USalign'

In [23]:
def parse_tmscore_output(output):
    # Extract TM-score based on length of reference structure (second)
    tm_score_match = re.findall(r'TM-score=\s+([\d.]+)', output)[1]
    if not tm_score_match:
        raise ValueError('No TM score found')
    return float(tm_score_match)


def write_target_line(
    atom_name, atom_serial, residue_name, chain_id, residue_num, x_coord, y_coord, z_coord, occupancy=1.0, b_factor=0.0, atom_type='P'
) -> str:
    """
    Writes a single line of PDB format based on provided atom information.
    """
    return f'ATOM  {atom_serial:>5d}  {atom_name:<5s} {residue_name:<3s} {residue_num:>3d}    {x_coord:>8.3f}{y_coord:>8.3f}{z_coord:>8.3f}{occupancy:>6.2f}{b_factor:>6.2f}           {atom_type}\n'

In [24]:
def write2pdb(df: pd.DataFrame, xyz_id: str, target_path: str) -> int:
    resolved_cnt = 0
    with open(target_path, 'w') as target_file:
        for _, row in df.iterrows():
            x_coord = row[f'x_{xyz_id}']
            y_coord = row[f'y_{xyz_id}']
            z_coord = row[f'z_{xyz_id}']

            if x_coord > -1e17 and y_coord > -1e17 and z_coord > -1e17:
                resolved_cnt += 1
                target_line = write_target_line(
                    atom_name="C1'",
                    atom_serial=int(row['resid']),
                    residue_name=row['resname'],
                    chain_id='0',
                    residue_num=int(row['resid']),
                    x_coord=x_coord,
                    y_coord=y_coord,
                    z_coord=z_coord,
                    atom_type='C',
                )
                target_file.write(target_line)
    return resolved_cnt

In [25]:
def write_ground_truth_pdb(df: pd.DataFrame, target_path: str) -> int:
    """
    Writes ground truth coordinates to a PDB file.
    """
    resolved_cnt = 0
    with open(target_path, 'w') as target_file:
        for _, row in df.iterrows():
            x_coord = row['x_1_true']
            y_coord = row['y_1_true']
            z_coord = row['z_1_true']

            if x_coord > -1e17 and y_coord > -1e17 and z_coord > -1e17:
                resolved_cnt += 1
                target_line = write_target_line(
                    atom_name="C1'",
                    atom_serial=int(row['resid']),
                    residue_name=row['resname'],
                    chain_id='0',
                    residue_num=int(row['resid']),
                    x_coord=x_coord,
                    y_coord=y_coord,
                    z_coord=z_coord,
                    atom_type='C',
                )
                target_file.write(target_line)
    return resolved_cnt

In [26]:
def evaluate_single_weight(args):
    """
    Evaluates a single weight combination for ensembling two RNA structure predictions.
    
    Args:
        args (tuple): Contains weight, first_pred_df, second_pred_df, temporary_dir, and usalign_path
        
    Returns:
        tuple: weight and resulting score
    """
    weight, first_pred_df, second_pred_df, temp_dir, usalign_path = args
    
    # Create a copy of the first prediction dataframe
    ensemble_df = first_pred_df.copy()
    
    # For each prediction (1-5), blend the coordinates using the weight
    for pred_idx in range(1, 6):
        # Blend x coordinates
        ensemble_df[f'x_{pred_idx}'] = (1 - weight) * first_pred_df[f'x_{pred_idx}'] + weight * second_pred_df[f'x_{pred_idx}']
        # Blend y coordinates
        ensemble_df[f'y_{pred_idx}'] = (1 - weight) * first_pred_df[f'y_{pred_idx}'] + weight * second_pred_df[f'y_{pred_idx}']
        # Blend z coordinates
        ensemble_df[f'z_{pred_idx}'] = (1 - weight) * first_pred_df[f'z_{pred_idx}'] + weight * second_pred_df[f'z_{pred_idx}']
    
    # Create unique filenames for this process to avoid conflicts in parallel processing
    process_id = os.getpid()
    native_pdb = f'{temp_dir}/native_{process_id}.pdb'
    predicted_pdb = f'{temp_dir}/predicted_{process_id}.pdb'
    
    # Evaluate using TM-score
    score = score_ensemble(ensemble_df, native_pdb, predicted_pdb, usalign_path)
    
    return weight, score

In [27]:
from tqdm import tqdm

def score_ensemble(ensemble_df, native_pdb='native.pdb', predicted_pdb='predicted.pdb', usalign_path=None):
    """
    Scores the ensemble predictions against the ground truth using TM-score.
    
    Args:
        ensemble_df (pd.DataFrame): DataFrame with predicted coordinates and ground truth
        native_pdb (str): Path to write the native PDB file
        predicted_pdb (str): Path to write the predicted PDB file
        usalign_path (str): Path to the USalign executable
        
    Returns:
        float: Average TM-score across all targets
    """
    if usalign_path is None:
        usalign_path = '/kaggle/working/USalign'
    
    from contextlib import contextmanager
    
    @contextmanager
    def no_stdout():
        old_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
        try:
            yield
        finally:
            sys.stdout = old_stdout
    
    # Extract target_id from ID (target_resid)
    ensemble_df['target_id'] = ensemble_df['ID'].apply(lambda x: x.split('_')[0])
    
    results = []
    # Iterate through each target_id and compute TM-score
    unique_targets = ensemble_df['target_id'].unique()
    for target_id in tqdm(unique_targets):
        group_data = ensemble_df[ensemble_df['target_id'] == target_id]
        
        # Write ground truth PDB once per target
        with no_stdout():
            resolved_cnt_truth = write_ground_truth_pdb(group_data, native_pdb)
        
        if resolved_cnt_truth == 0:
            continue  # Skip if no ground truth coordinates
        
        target_id_scores = []
        for pred_cnt in range(1, 6):
            # Write predicted PDB
            with no_stdout():
                resolved_cnt_pred = write2pdb(group_data, pred_cnt, predicted_pdb)
            
            if resolved_cnt_pred > 0:
                try:
                    command = f'{usalign_path} {predicted_pdb} {native_pdb} -atom " C1\'"'
                    usalign_output = os.popen(command).read()
                    tm_score = parse_tmscore_output(usalign_output)
                    target_id_scores.append(tm_score)
                except Exception as e:
                    # Silent error handling during optimization
                    continue
        
        if target_id_scores:
            results.append(max(target_id_scores))
    
    # Return average TM-score across all targets
    if not results:
        return 0.0
    return float(sum(results) / len(results))

In [28]:
def hill_climbing_optimizer(experiments_list, weight_step=0.05, max_iterations=1, num_cores=None):
    """
    Performs hill climbing optimization to find the best ensemble weights for RNA structure predictions.
    
    Args:
        experiments_list (list): List of dataframes with predictions
        weight_step (float): Step size for weight exploration
        max_iterations (int): Maximum number of iterations
        num_cores (int): Number of CPU cores to use for parallelization
        
    Returns:
        tuple: Dictionary of weights, best score, and final ensemble dataframe
    """
    # Set up USalign
    usalign_path = setup_usalign()
    
    if num_cores is None:
        num_cores = max(1, multiprocessing.cpu_count() - 1)
    
    # Create temporary directory for parallel processing
    temp_dir = '/tmp/rna_ensemble'
    os.makedirs(temp_dir, exist_ok=True)
    
    # Start with the first experiment as baseline
    best_ensemble_df = experiments_list[0].copy()
    print(f"First model in ensemble : {best_ensemble_df.attrs['name']}")
    remaining_experiments = experiments_list[1:].copy()
    
    # Dictionary to store weights for each experiment
    model_weights = {}
    
    # Define the weight range to explore
    weights_range = np.arange(0.0, 1.0 + weight_step, weight_step)
    weights_range = np.arange(-0.50, 0.51, 0.01)
    print(f"Number of weights testing : {len(weights_range)}")
    
    # Get initial score
    initial_score = score_ensemble(best_ensemble_df, usalign_path=usalign_path)
    best_score = initial_score
    print(f"Initial score: {best_score:.6f}")
    
    iteration = 0
    while remaining_experiments and iteration < max_iterations:
        iteration += 1
        print(f"\nIteration {iteration}/{max_iterations}")
        
        best_iteration = {
            'index': -1,
            'weight': 0,
            'score': best_score,
            'df': None
        }
        
        # Try each remaining model
        for i, model_df in enumerate(remaining_experiments):
            try:
                print(f"Testing model {i+1}/{len(remaining_experiments)}")
                
                # Prepare arguments for parallel processing
                args_list = [(weight, best_ensemble_df, model_df, temp_dir, usalign_path) 
                             for weight in weights_range]
                
                # Process weights in parallel
                with ProcessPoolExecutor(max_workers=num_cores) as executor:
                    futures = [executor.submit(evaluate_single_weight, args) for args in args_list]
                    
                    # Process results as they complete
                    for future in tqdm(as_completed(futures), total=len(futures), 
                                      desc=f"Testing weights for model {model_df.attrs['name']}"):
                        weight, new_score = future.result()
                        
                        # Update best if improved
                        if new_score > best_iteration['score']:
                            # Create the ensemble for this weight
                            tmp_ensemble_df = best_ensemble_df.copy()
                            for pred_idx in range(1, 6):
                                tmp_ensemble_df[f'x_{pred_idx}'] = (1 - weight) * best_ensemble_df[f'x_{pred_idx}'] + weight * model_df[f'x_{pred_idx}']
                                tmp_ensemble_df[f'y_{pred_idx}'] = (1 - weight) * best_ensemble_df[f'y_{pred_idx}'] + weight * model_df[f'y_{pred_idx}']
                                tmp_ensemble_df[f'z_{pred_idx}'] = (1 - weight) * best_ensemble_df[f'z_{pred_idx}'] + weight * model_df[f'z_{pred_idx}']
                            
                            best_iteration.update({
                                'index': i,
                                'weight': weight,
                                'score': new_score,
                                'df': tmp_ensemble_df.copy(),
                                'model_added' : model_df.attrs["name"]
                            })
                
                # Clean up
                gc.collect()
                
            except Exception as e:
                print(f"Error processing model {i+1}: {str(e)}")
                continue
        
        # Check if we found an improvement
        if best_iteration['index'] == -1:
            print("No improvement found, stopping")
            break
            
        # Update ensemble with best model found
        best_score = best_iteration['score']
        best_ensemble_df = best_iteration['df'].copy()
        
        # Add weight to the selected model and remove it from remaining
        selected_model = remaining_experiments.pop(best_iteration['index'])
        model_weights[f"model_{best_iteration['model_added']}"] = best_iteration['weight']
        
        print(f"Added model {best_iteration['model_added']} with weight {best_iteration['weight']:.4f}, New score: {best_iteration['score']:.6f}")
        
    print("\nOptimization complete!")
    print(f"Initial score: {initial_score:.6f}")
    print(f"Final score: {best_score:.6f}")
    print(f"Improvement: {best_score - initial_score:.6f} ({(best_score - initial_score) / initial_score * 100:.2f}%)")
    print(f"Model weights: {model_weights}")
    
    # Clean up temporary directory
    for filename in os.listdir(temp_dir):
        os.remove(os.path.join(temp_dir, filename))
    os.rmdir(temp_dir)
    
    return model_weights, best_score, best_ensemble_df

In [29]:
# Create list of experiment dataframes
experiments_list = [exp_1_oof, exp_2_oof, exp_3_oof, exp_4_oof]

# Run the hill climbing optimizer
model_weights, best_score, final_ensemble = hill_climbing_optimizer(
    experiments_list=experiments_list,
    weight_step=0.1,
    max_iterations=10,
    num_cores=4
)

USalign binary copied and made executable
First model in ensemble : PROTEINX
Number of weights testing : 101


100%|██████████| 143/143 [00:36<00:00,  3.89it/s]

Initial score: 0.435888

Iteration 1/10
Testing model 1/3



Testing weights for model RINET1: 100%|██████████| 101/101 [27:40<00:00, 16.44s/it]

Testing model 2/3



Testing weights for model RINET0: 100%|██████████| 101/101 [27:55<00:00, 16.59s/it]

Testing model 3/3



Testing weights for model RINET2: 100%|██████████| 101/101 [28:07<00:00, 16.71s/it]

No improvement found, stopping

Optimization complete!
Initial score: 0.435888
Final score: 0.435888
Improvement: 0.000000 (0.00%)
Model weights: {}


In [30]:
model_weights, best_score, final_ensemble

({},
 0.43588825174825163,
               ID resname  resid     x_1     y_1     z_1     x_2     y_2  \
 0       1FFK_9_1       U      1   6.074 -27.246  -7.244   1.181 -28.209   
 1       1FFK_9_2       U      2   5.920 -24.237  -2.370   3.976 -23.706   
 2       1FFK_9_3       A      3   0.383 -21.406  -1.219  -1.625 -18.410   
 3       1FFK_9_4       G      4   7.584 -21.293   3.046   7.512 -19.914   
 4       1FFK_9_5       G      5  11.798 -17.503   4.719  12.465 -17.354   
 ...          ...     ...    ...     ...     ...     ...     ...     ...   
 19201  8Z1F_T_62       U     62   2.582  -5.014   5.244   1.867  -5.770   
 19202  8Z1F_T_63       A     63   4.981  -6.225   9.920   5.965  -8.577   
 19203  8Z1F_T_64       C     64   4.950  -7.050  15.243   8.126 -12.665   
 19204  8Z1F_T_65       C     65   2.320  -7.998  19.906   7.777 -17.613   
 19205  8Z1F_T_66       A     66  -2.465 -10.161  22.057   5.057 -22.346   
 
           z_2     x_3  ...     x_4     y_4     z_4     x_5

In [31]:
# Save the final ensemble
final_ensemble.to_csv('final_ensemble.csv', index=False)